## Outils pour la manipulation d'images et librairies.


In [126]:
import PIL
from PIL import Image
import numpy as np
import scipy as sp
import os
from math import log10, sqrt

def load(filename):
    toLoad= Image.open(filename)
    return np.asarray(toLoad)


def psnr(original, compressed):
    mse = np.mean((original - compressed) ** 2)
    max_pixel = 255.0
    psnr = 20 * log10(max_pixel / sqrt(mse))
    return psnr

def dct2(a):
    return sp.fft.dct( sp.fft.dct( a, axis=0, norm='ortho' ), axis=1, norm='ortho' )

def idct2(a):
    return sp.fft.idct( sp.fft.idct( a, axis=0 , norm='ortho'), axis=1 , norm='ortho')



## Normalisation de l'image (YCbCr et padding)

In [127]:

#question 1 : 
#fonctions 1 et 2 données en TD 

def RGB_YCbCr (mat) :
#permet une conversion RGB à YCbCr
    Mat_YCbCr = np.empty(mat.shape)
    for i in range(mat.shape[0]):
        for j in range(mat.shape[1]):
            R = 0.299*mat[i,j,0]+0.587*mat[i,j,1]+0.114*mat[i,j,2]
            Cb = -0.1687*mat[i,j,0]-0.34414*mat[i,j,1]+0.5*mat[i,j,2]+128
            Cr = 0.5*mat[i,j,0]-0.4187*mat[i,j,1]-0.0813*mat[i,j,2]+128
            Mat_YCbCr[i,j] = (R,Cb,Cr) # donne la valeur des pixels 
    return Mat_YCbCr
#Question 2
#permet l'inverse 
def YCbCr_RGB(mat):
    Mat_RGB =np.empty(mat.shape)
    for i in range(mat.shape[0]):
        for j in range(mat.shape[1]):
            R = mat[i,j,0]+ 1.402*mat[i,j,1]-128
            G = mat[i,j,0]-0.34414* (mat[i,j,1]-128) - 0.71414*(mat[i,j,2]-128)
            B = mat[i,j,0]+1.772*mat[i,j,1]-128
            Mat_RGB[i,j] = (R,G,B)
    return Mat_RGB

#autres fonctions donneés en TD
def Y(mat):
    MatY=np.empty(mat.shape[0],mat.shape[1],1)
    for i in range (mat.shape[0]):
        for j in range (mat.shape[1]):
            MatY[i,j]=0.299*mat[i,j,0]+0.587*mat[i,j,1]+0.114*mat[i,j,2]
    return MatY 
           
def Cb(mat):
    MatCb=np.empty(mat.shape[0],mat.shape[1],1)
    for i in range (mat.shape[0]):
        for j in range (mat.shape[1]):
            MatCb=-0.1687*mat[i,j,0]-0.34414*mat[i,j,1]+0.5*mat[i,j,2]+128
    return MatCb        

def Cr(mat):
    MatCr=np.empty(mat.shape[0],mat.shape[1],1)
    for i in range (mat.shape[0]):
        for j in range (mat.shape[1]):
            MatCr= 0.5*mat[i,j,0]-0.4187*mat[i,j,1]-0.0813*mat[i,j,2]+128
    return MatCR        


#Question 3

def padding(mat):
    lignes,colonnes=mat.shape#on recupère le nombre de colonnes[1] et de lignes[0]
    nb_tot_l =lignes#variable qui va nous permettre de savoir si le nombre de ligne est un multiple de 8
    nb_tot_c=colonnes#pareil mais pour les colonnes 
    while nb_tot_l % 8 !=0:#si le reste ne vaut pas 0 alors ce n'est pas un multiple de 8 
        nb_tot_l+=1 #par conséquant on rajoute +1 juste qu'a ce que le reste vale 0
    while nb_tot_c %8 !=0:
        nb_tot_c += 1
    matrice_finale=np.zeros((nb_tot_l,nb_tot_c))#les lignes ou colonnes ajouter ont pour valeurs 0
    matrice_finale[:lignes,:colonnes]=mat #on lui insère les valeurs de la matrice initiale        


#question 4
mat=np.array([[1,2,3,6],[3,5,10,0]])#matrice donnée dans le sujet pour utiliser pour tester la fonction     
def ss_echant(mat):
    mat2=np.empty((mat.shape[0],int(mat.shape[1]/2)))#on se sert de la matrice de base avec mat.shape[O] pour prendre 
    #les ligne(2) puis [1]on divise par deux les colonnes(4/2=2) pour avoir une matrice vide plus petite qui va acceuillir nos futurs élements 
    for i in range(mat2.shape[0]):#on prend lignes 
        for j in range(mat2.shape[1]):# ainsi que les colonnes
            a,b=mat[i][j*2],mat[i][j*2 + 1]#a et b sont les variables qui vont acceuillir les 2 valeurs voisines qui seront diviser à l'avenir on 
            #multiplie par 2 pour etre sur que ca aille de 2 en 2 et non de 1 en 1
            c=(a+b)/2
            mat2[i][j]=c#remplit la matrice mat2
    return mat2
print(ss_echant(mat))
#question 5  

def multiplication_mat(mat2):# fonction qui est censer doubler le nombre de colonnes
    new_shape = (mat2.shape[0], 2 * mat2.shape[1])#shape="forme" donc on donne une nouvelle forme à une matrice 
    new_matrix = np.zeros(new_shape, dtype=mat2.dtype)#permet de remplacer les nouvelles colonnes par des zeros on revient donc a la taille de la matrice initiale :)
    new_matrix[:, ::2] = mat2#les 0 vont de 2 en 2
    return new_matrix#renvoie la nouvelle matrice 









[[1.5 4.5]
 [4.  5. ]]


<>:28: SyntaxWarning: 'float' object is not callable; perhaps you missed a comma?
<>:28: SyntaxWarning: 'float' object is not callable; perhaps you missed a comma?
C:\Users\kelly\AppData\Local\Temp\ipykernel_24096\878958684.py:28: SyntaxWarning: 'float' object is not callable; perhaps you missed a comma?
  g = (255/219) * (mat[i,j,0] - 16) - (255/224)*1.772*(0.114/0.587)*(mat[i,j,1]-128)-(255/224)*1.402(0.299/0.587)*(mat[i,j,2]- 128)


## Découpage en blocs et compression

In [90]:
#Question 6
def decouper_en_blocs(mat):
    lignes, colonnes = mat.shape#recupere leurs valeurs 
    nb_l = lignes // 8
    nb_c = colonnes // 8
    blocs = []#le blocs qui va contenir 8x8 
    for i in range(nb_l):#0
        for j in range(nb_c):#0
            bloc = mat[i*8:(i+1)*8, j*8:(j+1)*8].copy()#".copy()"permet d'etre sur que l'on ne copie pas la meme partie de la matrice
            #permet de delimiter la matrice de base en matrices 8x8 on donne les "mesures " à insérer dans la variable Bloc
            blocs.append(bloc)

    return blocs

#question 7

def appliquer_dct2_sur_blocs(liste_blocs):#chaque bloque est une matrice carrée
    blocs_transformes = []#liste vide qui acceuillir les blocs transformés lors l'execution de la fonction
    for bloc in liste_blocs:
        bloc_transforme = dct2(bloc)#on applique la transformée dct2 a chaque bloque a l'aide la fonction donnée dans le sujet
        blocs_transformes.append(bloc_transforme)#ajoute tout les blocs transformées a la variable "blocs_transformes"

    return blocs_transformes


#Question 8

def mode0(Image):#fonction reliée à l'un des modes de compressions (0)
    Y = Image[:,:,0]
    Cb = Image[:,:,1]
    Cr = Image[:,:,2]
    Y = appliquer_dct2_sur_blocs(decouper_en_blocs(Y))
    Cb = appliquer_dct2_sur_blocs(decouper_en_blocs(Cb))
    Cr = appliquer_dct2_sur_blocs(decouper_en_blocs(Cr))
    return Y,Cb,Cr#on garde les blocs transformés comme ils étaient au départ 

def filtrage(blocs_transformes,seuil):#fonction qui implement le filtrage des coefficients
    blocs_filtres=[]#contiendra les blocs filtrés
    for bloc in blocs_transformes:
        bloc_filtrer=np.where(np.abs(bloc)<seuil, 0,bloc)
        blocs_filtres.append(bloc_filtrer)
    return blocs_filtres 

def mode2(imageYCbCr, seuil):
    Y = imageYCbCr[:,:,0]
    Cb = ss_echant(imageYCbCr[:,:,1])
    Cr = ss_echant(imageYCbCr[:,:,2])
    Y = appliquer_dct2_sur_blocs(decouper_en_blocs(Y))
    Cb = appliquer_dct2_sur_blocs(decouper_en_blocs(Cb))
    Cr = appliquer_dct2_sur_blocs(decouper_en_blocs(Cr))
    for i in range(len(Y)):
        for j in range(len(Y[0])):
            for k in range(8):
                if Y[i][j][k] < seuil:
                    Y[i][j][k] = 0
    for i in range(len(Cb)):
        for j in range(len(Cb[0])):
            for k in range(8):
                if Cb[i][j][k] < seuil:
                    Cb[i][j][k] = 0
    for i in range(len(Cr)):
        for j in range(len(Cr[0])):
            for k in range(8):
                if Cr[i][j][k] < seuil:
                    Cr[i][j][k] = 0#on analyse tout les pixels de l'image
    return Y,Cb,Cr   

#Question 9

def compress(imageRGB,mode):
    if mode == "mode0":
        return mode0(imageRGB)
    elif mode == "mode1":
        seuil = int(input("entrer un seuil : "))
        return filtrage(imageRGB, seuil)
    elif mode == "mode2":
        seuil = int(input("entrer un seuil : "))
        return mode2(imageRGB, seuil)

## Écriture dans un fichier

In [91]:

#question 10
def ficher_ecriture(chemin, mat, rle) :
    f= open(chemin,"w")
    f.write("SJPG\n") 
    f.write(str(mat.shape[0])+"\n")
    f.write(str(mat.shape[1])+"\n")
    if rle:
        f.write("RLE\n") 
    else:
        f.write("NORLE\n") 

#question 11 : 

def bloc_y(valeurs):
    with open("bloc_y.txt", "w") as fichier:
        for valeur in valeurs:
            fichier.write(int(valeur) + "\n")

def bloc_Cb(valeurs):
    with open("bloc_Cb.txt", "w") as fichier:
        for valeur in valeurs:
            fichier.write(int(valeur) + "\n")  

def bloc_Cr(valeurs):
    with open("bloc_Cr.txt", "w") as fichier:
        for valeur in valeurs:
            fichier.write(int(valeur) + "\n")                  

## Décompression

In [8]:
#question 13
def décompression(Blocs_Y,Blocs_Cb,Blocs_Cr):
    hauteur_bloc,largeur_bloc=Blocs_Y[0][0].shape
    hauteur_image = hauteur_bloc*len(Blocs_Y)
    largeur_image = largeur_bloc*len(Blocs_Y[0])#hauteur et largeur permer le calcul de la taille de l'image finale RGB
    matrice_Y=np.zeros((hauteur_image,largeur_image))
    matrice_Cb=np.zeros((hauteur_image,largeur_image))
    matrice_Cr=np.zeros((hauteur_image,largeur_image))#matrices vides crées pour les canaux Y Cb Cr
    for i in range(len(Blocs_Y)):#avec la bouclle for on reconstruit chaque ca nal à partir des blocsF
        for j in range(len(Blocs_Y[0])):
            matrice_Y[i*hauteur_bloc:(i+1)*hauteur_bloc,j*largeur_bloc]=idct2(Blocs_Y[i][j])
            matrice_Cb[i*hauteur_bloc:(i+1)*hauteur_bloc,j*largeur_bloc]=idct2(Blocs_Cb[i][j])
            matrice_Cr[i*hauteur_bloc:(i+1)*hauteur_bloc,j*largeur_bloc]=idct2(Blocs_Cr[i][j])

    image_RGB=np.dstack((matrice_Y,matrice_Cb,matrice_Cr))
    return image_RGB

#Question 14 



    

## Tests 

In [137]:
test = load("test.png")
Image.fromarray(test,'RGB').show()


